In [1]:
import os
import re
import pymysql
import numpy as np
from collections import defaultdict
from shapely.geometry import Polygon
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore", UserWarning)

# Detectron2 Stuff
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities"
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [2]:
annot_paths = '/scratchh/home/anmol/cotton-data/splits/gold/annot/'
src = '/scratchh/home/anmol/cotton-data/splits/gold/split_v2/trap/'
image_path_src = '/scratchh/home/anmol/cotton-data/splits/gold/images'

In [3]:
annotations_files = os.listdir(annot_paths)

#### Get Images

In [7]:
image_splits = dict()
exceptions = []
for mode in ['train', 'val', 'test']:
    image_splits[mode] = []
    file_list = list(np.genfromtxt(os.path.join(src, f'{mode}.txt'), dtype='str'))
    for file in tqdm(file_list):
        annot_path = os.path.join(annot_paths, f'{file}.txt')
        annot_array = np.genfromtxt(annot_path)
        if annot_array.size > 0:
            if len(annot_array.shape) == 1:
                file_dict = {
                    'path' : os.path.join(image_path_src, f'{file}.jpg'),
                    'id' : file,
                    'label' : [annot_array[0]],
                    'bounding_boxes' : list(annot_array[1:]),
                }
            else:
                file_dict = {
                    'path' : os.path.join(image_path_src, f'{file}.jpg'),
                    'id' : file,
                    'label' : list(annot_array[:, 0]),
                    'bounding_boxes' : [list(l) for l in list(annot_array[:, 1:])]
                }
        else:
            file_dict = {
                'path' : os.path.join(image_path_src, f'{file}.jpg'),
                'id' : file,
                'label' : [],
                'bounding_boxes' : []
            }
        image_splits[mode].append(file_dict)

100%|██████████| 808/808 [00:01<00:00, 790.84it/s]


#### Set Up Data for Detectron

In [8]:
from detectron2.structures import BoxMode
from detectron2.data import MetadataCatalog

In [11]:
def get_pest_data(data_list):
    dataset_dicts = []
    for data_point in data_list:
        record = {}
        
        filename = data_point['path']
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = data_point['id']
        record["height"] = height
        record["width"] = width
      
        labels = data_point['label']
        bounding_boxes = data_point['bounding_boxes']
        objs = []
        for label, bounding_box in zip(labels, bounding_boxes):
            obj = {
                "bbox": [bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3]],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": label,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [13]:
for d in ["train", "val", "test"]:
    DatasetCatalog.register("pest_" + d, lambda d=d: get_pest_data(image_splits[d]))
    MetadataCatalog.get("pest_" + d).set(thing_classes=["American Ballworm", "Pink Ballworm"])

In [ ]:
# from detectron2.engine import DefaultTrainer

# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.DATASETS.TRAIN = ("pest_train",)
# cfg.DATASETS.TEST = ()
# cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
# cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
# cfg.SOLVER.STEPS = []        # do not decay learning rate
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# # NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg) 
# trainer.resume_or_load(resume=False)
# trainer.train()

[05/20 20:13:23 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res